# GNN for Parity Games Experiments

## Raw Training Data Creation

**07 July 2022**

- Num games: 3000
- Graph size (N) range: [10,200]
- Relative outdegree $(\frac{d_i}{N})$ range: [0.01,0.5]
- Priority range: [0,N]

In [1]:
num_graphs = 3000
min_n = 10
max_n = 200
min_rod = 0.01
max_rod = 0.5

games_dir = 'games'
solutions_dir = 'solutions'
pgsolver_base = 'pgsolver' # Path to compiled base dir of https://github.com/tcsprojects/pgsolver 

In [2]:
#import game_generator as gg

#gg.create_games_and_solutions(num_graphs, min_n, max_n, min_rod, max_rod, games_dir, solutions_dir, pgsolver_base)

## Prepare Training

In [3]:
from torch_geometric.loader import DataLoader
from parity_game_dataset import ParityGameDataset
import math

data = ParityGameDataset('pg_data_20220708', 'games', 'solutions')

# Use first 70% of the data for training
split_index = math.floor(0.7 * len(data))
train_data = data[:split_index]
test_data = data[split_index:]

train_loader = DataLoader(train_data, batch_size=5, shuffle=True)
test_loader = DataLoader(test_data, batch_size=5, shuffle=False)



**Training Parameters**

- optimizer: Adam
- Learning rate: 0.001
- Loss: Cross Entropy
- Epochs: 1-100

## Prepare model

**Parameters**

- Iterations: 10
- ...

In [62]:
from importlib import reload
import parity_game_network as pn
#from parity_game_network import ParityGameNetwork
reload(pn)
model = pn.ParityGameNetwork(256, 256, 10)
#model = pn.ParityGameNetwork(128, 128, 5)

In [63]:
model

ParityGameNetwork(
  (core): GAT(3, 256, num_layers=10)
  (node_classifier): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
  (edge_classifier): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)

# Run Training Loop

In [64]:
import wandb
wandb.init(project='gnn_parity_game_solver')
config = wandb.config
config.learning_rate = 0.001

wandb.watch(model, log='all')

Acc_Acc_Nodes,▁
Epoch,▁
Test_Acc_Edges,▁
Test_Acc_Nodes,▁
Train_Acc_Edges,▁
loss,█▄▂▁▁▁▁▁
variance,▁▃▆█▇█▇▆
Acc_Acc_Nodes,0.59897
Epoch,1
Test_Acc_Edges,0.96849
Test_Acc_Nodes,0.59813


[]

In [65]:
import torch
import numpy as np
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion_nodes = torch.nn.CrossEntropyLoss()
criterion_edges = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.1,0.9]))

def train():
    running_loss = 0.
    i = 0
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        i += 1
        optimizer.zero_grad()  # Clear gradients.
        out_nodes, out_edges = model(data.x, data.edge_index)  # Perform a single forward pass.
        
        # Most edges do not belong to a winning strategy and thus the data is extemely imbalanced. The model will probably learn that predicting always "non-winning" for each edge 
        # yields reasonable performance. To avoid this, approximately as many non-winning strategy edges are sampled as there are winning edges.
        #edge_selection = (torch.rand(data.y_edges.shape[0]) > 0.7) | (data.y_edges == 1)
        loss = criterion_nodes(out_nodes, data.y_nodes) + criterion_edges(out_edges, data.y_edges) # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        
        running_loss += loss.item()
        if i % 50 == 0:
            last_loss = running_loss / 50 # loss per batch
            wandb.log({'loss': last_loss, 'variance': torch.var(out_nodes[:,1])})
            running_loss = 0.
            
def test(loader):
     model.eval()

     correct_nodes = 0
     correct_edges = 0

     for data in loader:  # Iterate in batches over the training/test dataset.
        out_nodes, out_edges = model(data.x, data.edge_index)  
        pred_nodes = out_nodes.argmax(dim=1)  # Use the class with highest probability.
        pred_edges = out_edges.argmax(dim=1)  # Use the class with highest probability.
        correct_nodes += (pred_nodes == data.y_nodes).sum() / len(pred_nodes)  # Check against ground-truth labels.
        correct_edges += (pred_edges == data.y_edges).sum() / len(pred_edges)
     return (correct_nodes / len(loader), correct_edges / len(loader))  # Derive ratio of correct predictions.

for epoch in range(1, 2):
    train()
    train_acc_nodes, train_acc_edges = test(train_loader)
    test_acc_nodes, test_acc_edges = test(test_loader)
    wandb.log({'Epoch': epoch, 'Acc_Acc_Nodes': train_acc_nodes, 'Test_Acc_Nodes': test_acc_nodes, 'Train_Acc_Edges': train_acc_edges, 'Test_Acc_Edges': test_acc_edges}) 
    print(f'Epoch: {epoch:03d}, Train Acc Nodes: {train_acc_nodes:.4f}, Test Acc Nodes: {test_acc_nodes:.4f}, Train Acc Edges: {train_acc_edges:.4f}, Test Acc Edges: {test_acc_edges:.4f}')

Epoch: 001, Train Acc Nodes: 0.9834, Test Acc Nodes: 0.9823, Train Acc Edges: 0.9686, Test Acc Edges: 0.9683


*Saving the following model:*
- Performance: Epoch: 001, Train Acc Nodes: 0.9832, Test Acc Nodes: 0.9824, Train Acc Edges: 0.9687, Test Acc Edges: 0.9685

In [16]:
print(model)

ParityGameNetwork(
  (core): GAT(3, 256, num_layers=10)
  (node_classifier): Sequential(
    (0): Linear(in_features=256, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
  (edge_classifier): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)


In [73]:
torch.save(model.state_dict(), 'GAT_pg_solver_20220914.pth')

## Model application example

In [74]:
it = iter(test_loader)
next(it)
example_game = next(it)
out_nodes, out_edges = model(example_game.x, example_game.edge_index) 
pred_nodes = out_nodes.argmax(dim=1)
pred_edges = out_edges.argmax(dim=1)

### Output 1: Winning reagions of players 0 and 1

**Predicted regions**

In [75]:
pred_nodes

tensor([0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
        0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,

**Actual winning regions (Calculated by pgsolver)**

In [76]:
example_game.y_nodes

tensor([0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,

### Output 2: Winning strategies

A **1** means that the edge belongs to a winning strategy

**Predicted winning strategy**

In [77]:
example_game.edge_index[:,example_game.y_edges == 1]

tensor([[  0,   2,   3,   4,   6,   7,   8,   9,  10,  11,  12,  13,  15,  16,
          17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  28,  29,  31,  33,
          34,  35,  36,  37,  40,  41,  42,  44,  45,  46,  47,  48,  49,  50,
          51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
          65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
          79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
          93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106,
         107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
         121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 135, 136,
         137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150,
         151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
         165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
         179, 180, 181, 182, 183, 184, 185, 186, 187

**Winning strategy from pgsolver**

In [78]:
example_game.edge_index[:,pred_edges == 1]

tensor([[  9,   9,  20, 101, 117, 117, 138, 138, 138, 138, 292, 292, 292],
        [ 12,  16,  16, 150,  51, 102,  71, 125, 153, 178, 250, 309, 334]])

In [72]:
max(pred_edges)

tensor(1)

In [43]:
example_game.y_edges.shape[0]

17852